# Euler–Maclaurin 2D formula

In [1]:
def num_int(f, a, b):
    from scipy import integrate

    try:
        integral = integrate.quad(f, a, b)

        result = integral[0]
        return result

    except Exception as ex:

        print ("Exception ex = ", str(ex))
        print ("f = ", f)
        try:
            #print ("integrand = ", get_integrand_view(f))
            print ("integrand = ", f(x))
        except Exception as ex2:
            print ("Exception ex2 = ", ex2)

        print ("a, b = ", a, ",", b)
        try:
            print ("f(a) = ", f(a))
        except:
            pass
        try:
            print ("f(b) = ", f(b))
        except:
            pass

        import inspect
        stack = inspect.stack()
        for frame in stack:
            func_name = frame[3]
            print ("func_name = ", func_name)
        
        raise ex

In [2]:
logging = False

calc_R_p=True

use_int_R_p_j=True
change_R_p_order=True
use_int_R_p_shifted_first=True
use_num_int = False

def my_euler_maclaurin1(f,symb,a,b,p, f_diff_symb_p=None):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")


    sum_dfdx_a_bernoullis = sum([(f.diff(symb,k-1))*(bernoulli(k)/factorial(k)) for k in range(1,1+p)])
    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    if True:
        B = lambda x, p : bernoulli_polynomial(x, p)
        P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
        fact = lambda n : factorial(n)
    else:
        B = function("B")(x,p)
        P = function("P")(x,p)
        fact = function("fact")(n)
        
    if True == calc_R_p:
        # Can't calculate derivative wrt j + u.
        #int_R_p = lambda j : integral(f.diff(symb,p).subs(symb == u+j) * bernoulli_polynomial(u,p)/factorial(p), (u,0,1))
        def int_R_p(f,symb,j):
            v = var("v")            
            if None == f_diff_symb_p:
                deriv = f.diff(symb,p)
            else:
                deriv = f_diff_symb_p
            if logging:
                print("deriv", deriv)
            deriv_s = deriv.subs(symb == v+j)
            if logging:
                print("deriv_s", deriv_s)
            int_f_diff_B_fact = integral(deriv_s * bernoulli_polynomial(v,p)/factorial(p), (v,0,1))
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)
            return int_f_diff_B_fact
        
        def int_R_p_shifted_first(f,symb,j):
            v = var("v")
            fn_shifted = f.subs(symb == v + j)
            if logging:
                print("fn_shifted", fn_shifted)
            deriv = fn_shifted.diff(v,p)
            if logging:
                print("deriv", deriv)
            int_f_diff_B_fact = integral(deriv * B(x=v,p=p)/fact(n=p), (v,0,1))
    
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)

            return int_f_diff_B_fact

        if logging:
            print("int_R_p(j = a)", int_R_p(f,symb,j = a))
            
        if use_int_R_p_j:
            v, j = var("v, j")
            assume(j>0)
            if change_R_p_order:
                # Change order of summation and integration
                if use_int_R_p_shifted_first:
                    R_p = (-1)^(p+1)*integral(sum(f.subs(symb == v+j).diff(v,p)*B(x=v,p=p)/fact(n=p), \
                                                  j, a, b-1), \
                                              (v,0,1))
                else:
                    if None == f_diff_symb_p:
                        f_diff_symb_p = f.diff(symb,p)
                    R_p = (-1)^(p+1)*integral(sum(f_diff_symb_p.subs(symb == v+j)*B(x=v,p=p)/fact(n=p), \
                                                  j, a, b-1), \
                                              (v,0,1))
            else:
                if use_int_R_p_shifted_first:
                    R_p = (-1)^(p+1)*sum(int_R_p_shifted_first(f,symb,j=j), j, a, b-1)
                else:
                    R_p = (-1)^(+1)*sum(int_R_p(f,symb,j=j), j, a, b-1)
                
        else:
            if None == f_diff_symb_p:
                f_diff_symb_p = f.diff(symb,p)
            if not use_num_int:
                R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=True)
            else:
                R_p = (-1)^(p+1)*num_int(lambda symb : f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b)
                

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
    
    s = int_fn_a_b - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    if True == calc_R_p:
        s += R_p
        
    if False:
        print("int_fn_a_b", int_fn_a_b)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis(a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis(b))
        if True == calc_R_p:
            print("int_R_p(j = a)", int_R_p(j = a))
            print("R_p", R_p)
        print("")
        print("s",s)
    return s

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+{\frac {f(a)+f(b)}{2}}+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{2k}}{(2k)!}}(f^{(2k-1)}(b)-f^{(2k-1)}(a))+R_{p},}$$

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))+R_{p},}$$

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(v+j){\frac {B_{p}(v)}{p!}}\,dv.}$$

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$${\displaystyle R_{p}=(-1)^{p+1}\int _{a}^{b}f^{(p)}(x){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [3]:
# https://www.journals.vu.lt/LMJ/article/view/20600/19701
# ФОРМУЛА СУММИРОВАНИЯ ЭЙЛЕРА—МАКЛОРЕНА ДЛЯ ФУНКЦИИ МНОГИХ ПЕРЕМЕННЫХ А.БИКЯЛИС
# VIII LIETUVOS MATEMATIKOS RINKINYS 4
# ЛИТОВСКИЙ МАТЕМАТИЧЕСКИЙ СБОРНИК
# 1968

# Далее,обе стороны этого равенства суммируем по всем целым m2 из интервала(z2,y2). Получаем

if True:
    B = lambda x, p : bernoulli_polynomial(x, p)
    P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
    fact = lambda n : factorial(n)
else:
    B = function("B")(x,p)
    P = function("P")(x,p)
    fact = function("fact")(n)

def integral_R_p(f,symb,jx,hold, f_diff_symb_p=None):
    vx = var("v_"+str(symb))
    if None == f_diff_symb_p:
        deriv = f.diff(symb,p)
    else:
        deriv = f_diff_symb_p
    if logging:
        print("deriv", deriv)

    deriv_subs = deriv.subs(symb == vx+jx)
    if logging:
        print("deriv_subs", deriv_subs)

    int_f_diff_B_fact = integral(deriv_subs * bernoulli_polynomial(vx,p)/factorial(p), (vx,0,1), hold=hold)
    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact

def integral_R_p_shifted_first(f,symb,jx,hold):
    vx = var("v_"+str(symb))
    fn_shifted = f.subs(symb == vx + jx)
    if logging:
        print("fn_shifted", fn_shifted)

    deriv = fn_shifted.diff(vx,p)
    if logging:
        print("deriv", deriv)

    int_f_diff_B_fact = integral(deriv * B(x=vx,p=p)/fact(n=p), (vx, 0, 1), hold=hold)

    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact
    
    
def euler_maclaurin_R_p(f,symb,a,b,p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")
    if use_int_R_p_j:
        jx = var('j_'+str(symb))
        assume(jx>0)
        
        if change_R_p_order:
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                R_p = (-1)^(p+1)*integral(sum(f.subs(symb == v+j).diff(v,p)*B(x=v,p=p)/fact(n=p), \
                                              j, a, b-1, hold=hold_sum), \
                                          (v,0,1), hold=hold_int)
            else:
                if None == f_diff_symb_p:
                    f_diff_symb_p = f.diff(symb,p)
                R_p = (-1)^(p+1)*integral(sum(f_diff_symb_p.subs(symb == v+j)*B(x=v,p=p)/fact(n=p), \
                                              j, a, b-1, hold=hold_sum), \
                                          (v,0,1), hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                term = (-1)^(p+1)*integral_R_p_shifted_first(f=f, symb=symb, jx=jx, hold=hold_int)
                R_p = sum(term, jx, a, b-1, hold=hold_sum)
            else:
                term = (-1)^(p+1)*integral_R_p(f=f, symb=symb, jx=jx, hold=hold_int, f_diff_symb_p=f_diff_symb_p)
                R_p = sum(term, jx, a, b-1, hold=hold_sum)
            return R_p, term
    else:
        if not use_num_int:
            if None == f_diff_symb_p:
                f_diff_symb_p = f.diff(symb,p)
            R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=hold_int)
        else:
            R_p = (-1)^(p+1)*num_int(lambda symb: f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b)
            
    return R_p

$$\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))$$

In [4]:
def sum_dfdx_bernoulis(f,symb,a,b,p):
    dfdx_a_bernoullis = []
    for k in range(1,1+p):
        dfdx_a_bernoullis += [(f.diff(symb,k-1))*(bernoulli(k)/factorial(k))]
        
    sum_dfdx_a_bernoullis = sum(dfdx_a_bernoullis)

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    s = - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    return s


In [5]:
def sum_dfdx_bernoulis_str(f,symb,a,b,p):
    sum_dfdx_a_bernoullis = ""
    for k in range(1,1+p):
        if k - 1 > 0:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str(f) + ").diff(" + str(symb) + ", " + str(k-1) + ")"
        else:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str(f) + ")"
            

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(a) + ")")
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(b) + ")")
    s = ""

    s += "- (" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(a) + ")"
    if Infinity != b:
        s += " + (" + sum_dfdx_a_bernoullis + ").subs(" + str(symb) + "==" + str(b) + ")"
    return s


In [6]:
# sumy_sumx = sum_dfdx_bernoulis(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def sum_dfdy_bernoulis_sum_dfdx_bernoulis_str(f, x, a_x, b_x, y, a_y, b_y, p):
    str_f = sum_dfdx_bernoulis_str(f(x,y), x, a_x, b_x, p)
    sum_dfdx_a_bernoullis = ""
    for k in range(1,1+p):
        if k - 1 > 0:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str_f + ").diff(" + str(y) + ", " + str(k-1) + ")"
        else:
            sum_dfdx_a_bernoullis += " + (" + str(bernoulli(k)/factorial(k)) + ") * (" + str_f + ")"

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(a_y) + ")")
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", "(" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(b_y) + ")")
    s = ""

    s += "- (" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(a_y) + ")"
    if Infinity != b_y:
        s += " + (" + sum_dfdx_a_bernoullis + ").subs(" + str(y) + "==" + str(b_y) + ")"
    return s

In [7]:
def definite_integral_derivative(f, x, a, b, diff_param, hold=True):
    from sage.symbolic.integration.integral import definite_integral

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = definite_integral(f.diff(diff_param), x, a, b, hold=hold)
    else:
        ans = SR.zero()

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += f.subs(x == b) * b.diff(diff_param)
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans -= f.subs(x == a) * a.diff(diff_param)

    return ans


In [8]:
def definite_integral_derivative_str(f, x, a, b, diff_param=None, hold=False, algorithm=None):

    if not x.has(diff_param):
        # integration variable != differentiation variable
        if algorithm is not None:
            ans = "integrate((" + str(f) + ").diff(" + str(diff_param) + "), " + str(x) + ", " + str(a) + ", " + str(b) + ", algorithm = "\""+ algorithm + ", hold = " + str(hold) + ")"
        else:
            ans = "definite_integral((" + str(f) + ").diff(" + str(diff_param) + "), " + str(x) + ", " + str(a) + ", " + str(b) + ", hold = " + str(hold) + ")"
    else:
        ans = ""

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += " + " + str(f.subs(x == b) * b.diff(diff_param))
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans += " - " + str(f.subs(x == a) * a.diff(diff_param))

    return ans

In [9]:
def definite_num_int_derivative_str(f, x, a, b, diff_param=None):

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = "num_int(lambda " + str(x)+ " : " + str(f.diff(diff_param)) + ", " + str(a) + ", " + str(b) + ")"
    else:
        ans = ""

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += " + " + str(f.subs(x == b) * b.diff(diff_param))
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans += " - " + str(f.subs(x == a) * a.diff(diff_param))

    return ans

$$\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(g(y)^{(k-1)}(b_y)-g(y)^{(k-1)}(a_y))$$
where 

$$g(y) = {\displaystyle R_{p}(y)=(-1)^{p+1}\sum_{j=a_x}^{b_x-1} \int _{0}^{1}\frac{\partial^{p}}{\partial x^{p}}f(v_x+j,y){\frac {B_{p}(v_x)}{p!}}\,dv_x.}$$
or

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$$g(y) = {\displaystyle R_{p}=(-1)^{p+1}\int _{a_x}^{b_x}\frac{\partial^{p}}{\partial x^{p}}f(x,y){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [10]:
#int_R_p = integral(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), (y, a_y, b_y), hold=True)

def integral_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("f(x,y)", f(x,y))
        if None != f_diff_x_p:
            print("f_diff_x_p", f_diff_x_p)
            print("f_diff_x_p(x,y)", f_diff_x_p(x,y))
        print("")
    if use_int_R_p_j:
        jx = var('j_x')
        assume(jx>0)
        
        if change_R_p_order:
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                int_R_p = (-1)^(p+1)*integral(integral(sum(f.subs(x == v+j).diff(v,p)*B(x=v,p=p)/fact(n=p), \
                                              j, a_x, b_x-1, hold=hold_sum), \
                                          (v,0,1), hold=hold_int), y, a_y, b_y, hold=hold_int)
            else:
                if None == f_diff_x_p:
                    f_diff_x_p = f.diff(x,p)
                int_R_p = (-1)^(p+1)*integral(integral(sum(f_diff_x_p.subs(x == v+j)*B(x=v,p=p)/fact(n=p), \
                                              j, a_x, b_x-1, hold=hold_sum), \
                                          (v,0,1), hold=hold_int), y, a_y, b_y, hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                int_int = (-1)^(p+1)*integral(integral_R_p_shifted_first(f=f, symb=x, jx=jx, hold=hold_int), y, a_y, b_y, hold=hold_int)
                int_R_p = sum(int_int, jx, a_x, b_x-1, hold=hold_sum)
            else:
                int_int = (-1)^(p+1)*integral(integral_R_p(f=f, symb=x, jx=jx, hold=hold_int), y, a_y, b_y, hold=hold_int)
                int_R_p = sum(int_int, jx, a_x, b_x-1, hold=hold_sum)
            return (int_R_p, int_int)
    else:
        if not use_num_int:
            if None == f_diff_x_p:
                f_diff_x_p = f.diff(x,p)
            int_R_p = (-1)^(p+1)*integral(integral(f_diff_x_p * P(x=x,p=p)/fact(n=p),(x,a_x,b_x),hold=hold_int), y, a_y, b_y, hold=hold_int)
        else:
            int_R_p = (-1)^(p+1)*num_int(lambda y : num_int(lambda x : f_diff_x_p(x,y) * P(x=x,p=p)/fact(n=p), a_x, b_x), a_y, b_y)
            
    return int_R_p

In [11]:
#sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def sum_dfdx_bernoulis_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p):
    if logging:
        print("sum_dfdx_bernoulis_euler_maclaurin_R_p", f, x, a_x, b_x, y, a_y, b_y, p)
    dfdx_a_bernoullis = []
    dfdx_a_bernoullis_terms = []
    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    
    for k in range(1,1+p):
        if 1 == k:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_1 = lambda y_symb, y_val, jx : (-1)^(p+1)*definite_integral(f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                                                      * B(x=vx,p=p)/fact(n=p),vx,0,1,hold=False) \
                                                                                .subs(y_symb == y_val)
                
                if logging:
                    print("term_1(y_symb=y, y_val=a_y,jx=a_x) =", term_1(y_symb=y, y_val=a_y,jx=a_x))
                R_p = lambda y_symb, y_val : symbolic_sum(expression=term_1(y_symb=y_symb, y_val=y_val, jx=jx), \
                                                                     v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, jx, i=i : \
                                            term_1(y_symb=y_symb, y_val=y_val, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                term_1 = lambda y_symb, y_val : definite_integral(f(x,y_symb).diff(x,p) * P(x=x,p=p)/fact(n=p), \
                                                                  x, a_x, b_x, hold=True) \
                                                                  .subs(y_symb == y_val)
                if logging:
                    print("term_1(y_symb=y, y_val=a_y) =", term_1(y_symb=y, y_val=a_y))
                R_p = lambda y_symb, y_val : (-1)^(p+1)*term_1(y_symb, y_val)
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p(y_symb=y, y_val=a_y))
            dfdx_a_bernoullis += [lambda y_symb, y_val, i=i : R_p(y_symb, y_val)*(bernoulli(k)/factorial(k))]
            
        elif 2 == k:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_2 = lambda y_symb, y_val, jx : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                        * B(x=vx,p=p)/fact(n=p), x=vx, a=0, b=1, diff_param=y_symb, hold=False) \
                                                                                .subs(y_symb == y_val)

                if logging:
                    print("term_2(y_symb=y, y_val=a_y,jx=a_x) =", term_2(y_symb=y, y_val=a_y,jx=a_x))

                R_p = lambda y_symb, y_val : symbolic_sum(expression=term_2(y_symb=y_symb, y_val=y_val, jx=jx), \
                                                                 v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, jx, i=i : \
                                            term_2(y_symb=y_symb, y_val=y_val, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                term_2 = lambda y_symb, y_val : definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                            * P(x=x,p=p)/fact(n=p), x=x, a=a_x, b=b_x, diff_param=y_symb) \
                                                  .subs(y_symb == y_val)

                if logging:
                    print("term_2(y_symb=y, y_val=a_y) =", term_2(y_symb=y, y_val=a_y))

                R_p = lambda y_symb, y_val : (-1)^(p+1)*term_2(y_symb, y_val)
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p(y_symb=y, y_val=a_y))
            dfdx_a_bernoullis += [lambda y_symb, y_val, i=i : R_p(y_symb, y_val)*(bernoulli(k)/factorial(k))]
        else:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_3 = lambda y_symb, y_val, jx : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                        * B(x=vx,p=p)/fact(n=p), x=vx, a=0, b=1, diff_param=y_symb, hold=False) \
                                            .diff(y_symb, k-2) .subs(y_symb == y_val)

                if logging:
                    print("term_3(y_symb=y, y_val=a_y,jx=a_x) =", term_3(y_symb=y, y_val=a_y,jx=a_x))
                R_p = lambda y_symb, y_val : symbolic_sum(expression=term_3(y_symb=y_symb, y_val=y_val, jx=jx), \
                                                                 v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, jx, i=i : \
                                            term_3(y_symb=y_symb, y_val=y_val, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                term_3 = lambda y_symb, y_val : definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                    * P(x=x,p=p)/fact(n=p), x=x, a=a_x, b=b_x, diff_param=y_symb) \
                                                    .diff(y_symb, k-2).subs(y_symb == y_val)

                if logging:
                    print("term_3(y_symb=y, y_val=a_y,jx=a_x) =", term_3(y_symb=y, y_val=a_y,jx=a_x))
                R_p = lambda y_symb, y_val : (-1)^(p+1)*term_3(y_symb, y_val)
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p(y_symb=y, y_val=a_y))
            dfdx_a_bernoullis += [lambda y_symb, y_val, i=i : R_p(y_symb, y_val)]

    if logging:
        print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
        print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

    sum_dfdx_a_bernoullis = lambda y_symb, y_val : sum([dfdx_a_bernoullis[i](y_symb, y_val) \
                                                      for i in range(len(dfdx_a_bernoullis))])
    
    sum_dfdx_a_bernoullis_terms = lambda y_symb, y_val, jx : sum([dfdx_a_bernoullis_terms[i](y_symb, y_val, jx) \
                                                               for i in range(len(dfdx_a_bernoullis_terms))])
    
    if logging:
        print("sum_dfdx_a_bernoullis =", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a_y) =", sum_dfdx_a_bernoullis(y_symb=y, y_val=a_y))
    
    s = - sum_dfdx_a_bernoullis(y_symb=y, y_val=a_y)
    if b_y is Infinity:
        s_terms = lambda jx : - sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=a_y, jx=jx)
    else:
        s += sum_dfdx_a_bernoullis(y_symb=y, y_val=b_y)
        s_terms = lambda jx : sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=b_y, jx=jx) - sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=a_y, jx=jx)
    return s, s_terms

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a_y}^{b_y-1} \int _{0}^{1}g^{(p)}(v_y+j){\frac {B_{p}(v_y)}{p!}}\,dv_y.}$$

$${\displaystyle P_{k}(y)=B_{k}(y-\lfloor y\rfloor ),}$$

$${\displaystyle R_{p}=(-1)^{p+1}\int _{a_y}^{b_y}g^{(p)}(y){\frac {P_{p}(y)}{p!}}\,dy.}$$

where

$$g(y) = {\displaystyle R_{p}(y)=(-1)^{p+1}\sum_{j=a_x}^{b_x-1} \int _{0}^{1}\frac{\partial^{p}}{\partial x^{p}}f(v_x+j,y){\frac {B_{p}(v_x)}{p!}}\,dv_x.}$$
or

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$$g(y) = {\displaystyle R_{p}=(-1)^{p+1}\int _{a_x}^{b_x}\frac{\partial^{p}}{\partial x^{p}}f(x,y){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [12]:
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p_diff_y_p):

    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    if use_int_R_p_j:
        vx,vy = var("v_x, v_y")
        jx,jy = var("j_x, j_y")
        assume(v_y+j_y>0)
        
        if None == f_diff_x_p_diff_y_p:
            f_diff_x_p_diff_y_p = f.diff(x,p).diff(y,p)

        R_p_R_p = (-1)^(p+1)*symbolic_sum(definite_integral( \
                               (-1)^(p+1)*symbolic_sum(definite_integral(f_diff_x_p_diff_y_p.subs(x == vx + jx)  \
                                                              * B(x=vx,p=p)/fact(n=p), \
                                 vx,0,1,hold=True), v=jx, a=a_x, b=b_x-1,hold=True) \
                                                                        .diff(y,p).subs(y == vy + jy)  \
                                                              * B(x=vy,p=p)/fact(n=p), \
                                 vy,0,1,hold=True), v=jy, a=a_y, b=b_y-1,hold=True)
    else:
        if not use_num_int:

            if None == f_diff_x_p_diff_y_p:
                f_diff_x_p_diff_y_p = f.diff(x,p)

            R_p_R_p = (-1)^(p+1)*definite_integral( \
                               (-1)^(p+1)*definite_integral(f_diff_x_p_diff_y_p  \
                                                              * P(x=x,p=p)/fact(n=p), \
                                 x, a_x, b_x, hold=True).diff(y,p)  \
                                                              * P(x=y,p=p)/fact(n=p), \
                                 y, a_y, b_y, hold=True)
        else:
            f_diff_x_p_diff_y_p_num_str = "f_diff_x_diff_y_num = lambda " + str(x) + ", " + str(y) + ":" + str(f(x,y).diff(x,p).diff(y,p))
            print("f_diff_x_p_diff_y_p_num_str=", f_diff_x_p_diff_y_p_num_str)                                                                          
            exec(preparse(f_diff_x_p_diff_y_p_num_str))
            R_p_R_p = (-1)^(p+1)*num_int(lambda y : \
                               (-1)^(p+1)*num_int(lambda x : f_diff_x_p_diff_y_p_num(x,y)  \
                                                              * P(x=x,p=p)/fact(n=p), \
                                 a_x, b_x)  \
                                                              * P(x=y,p=p)/fact(n=p), \
                                 a_y, b_y)
        
    return R_p_R_p

In [13]:
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p_num_int(f_diff_x_p_diff_y_p, x, a_x, b_x, y, a_y, b_y, p):
    R_p_R_p = (-1)^(p+1)*num_int(lambda y : \
                       (-1)^(p+1)*num_int(lambda x : f_diff_x_p_diff_y_p(x,y)  \
                                                      * P(x=x,p=p)/fact(n=p), \
                         a_x, b_x)  \
                                                      * P(x=y,p=p)/fact(n=p), \
                         a_y, b_y)
    return R_p_R_p

In [14]:
def euler_maclaurin1(f, symb, a, b, p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")
    #u, j = var("u, j")

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
    
    #s = int_fn_a_b + sum_dfdx_a_bernoullis(b) - sum_dfdx_a_bernoullis(a) + R_p
    s = int_fn_a_b + sum_dfdx_bernoulis(f,symb,a,b,p)
    
    if True == calc_R_p:
        R_p = euler_maclaurin_R_p(f, symb, a, b, p, f_diff_symb_p=f_diff_symb_p,hold_int=hold_int, hold_sum=hold_sum)
        if type(R_p) is tuple:
            s += R_p[0]
        else:
            s += R_p

    return s

In [15]:
from IPython.display import display, Math, Latex

In [16]:
a_x = var("n_x")
b_x = var("b_x")
a_y = var("n_y")
b_y = var("b_y")

assume(a_x, 'real')
assume(b_x, 'real')
assume(a_y, 'real')
assume(b_y, 'real')

assume(a_x, 'integer')
assume(b_x, 'integer')
assume(a_y, 'integer')
assume(b_y, 'integer')

assume(a_x > 0)
assume(b_x > 0)
assume(a_y > 0)
assume(b_y > 0)

x,y = var("x,y")

n = QQ['n'].0
j = QQ['j'].0

p = 4
p = 2
p = 1

F = function("F")(x, y)
print(F)

F(x, y)


In [17]:
ans = my_euler_maclaurin1(F(x,y),x,a_x,b_x,p, diff(F(x,y),x,p))
display(Math(latex(ans)))


Can't calculate derivative wrt j + v.


<IPython.core.display.Math object>

In [18]:
ans = euler_maclaurin1(F(x,y),x,a_x,b_x,p, diff(F(x,y),x,p))
display(Math(latex(ans)))

<IPython.core.display.Math object>

In [19]:
ans = my_euler_maclaurin1(F(x,y),x,a_x,b_x,p, diff(F(x,y),x,p))
display(Math(latex(ans)))


Can't calculate derivative wrt j + v.


<IPython.core.display.Math object>

In [20]:
ans = euler_maclaurin1(F(x,y),x,a_x,b_x,p, diff(F(x,y),x,p))
display(Math(latex(ans)))

<IPython.core.display.Math object>

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(u+j){\frac {B_{p}(u)}{p!}}\,du.}$$

In [21]:
inty = integral           (F(x,y),(y, a_y, b_y))
print("inty=$", latex(inty),"$")
display(Math(latex(inty)))

sumy = sum_dfdx_bernoulis (F(x,y), y, a_y, b_y,p)
print("sumy=$", latex(sumy),"$")
display(Math(latex(sumy)))

sumy = sum_dfdx_bernoulis_str (F(x,y), y, a_y, b_y,p)
print("sum_y=", sumy)
exec(preparse("sum_y=" + sumy))
print(preparse("sum_y=" + sumy))
print("sum_y=$", latex(sum_y),"$")
display(Math("sum_y=" + latex(sum_y)))

R_py = euler_maclaurin_R_p(F(x,y), y, a_y, b_y,p)
print("R_py=", R_py)
print("R_py=$", latex(R_py),"$")
display(Math(latex(R_py)))

inty=$ \int_{n_{y}}^{b_{y}} F\left(x, y\right)\,{d y} $


<IPython.core.display.Math object>

sumy=$ \frac{1}{2} \, F\left(x, n_{y}\right) $


<IPython.core.display.Math object>

sum_y= - ( + (-1/2) * (F(x, y))).subs(y==n_y)
sum_y=- ( + (-Integer(1)/Integer(2)) * (F(x, y))).subs(y==n_y)
sum_y=$ \frac{1}{2} \, F\left(x, n_{y}\right) $


<ipython-input-21-da37a57be299>:11: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  exec(preparse("sum_y=" + sumy))


<IPython.core.display.Math object>

R_py= integrate(sum(1/2*(2*v - 1)*D[1](F)(x, j + v), j, n_y, b_y - 1), v, 0, 1)
R_py=$ \int_{0}^{1} {\sum_{j=n_{y}}^{b_{y} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \mathrm{D}_{1}\left(F\right)\left(x, j + v\right)}\,{d v} $


<IPython.core.display.Math object>

In [22]:
x_int = integral           (F(x,y),(x, a_x, b_x))
print("x_int=$", latex(x_int),"$")
display(Math(latex(x_int)))

x_sum = sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p)
print("x_sum=$", latex(x_sum),"$")
display(Math(latex(x_sum)))

x_R_p = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, diff(F(x,y),x,p))
print("x_R_p=$", latex(x_R_p),"$")
display(Math(latex(x_R_p)))

x_int=$ \int_{n_{x}}^{b_{x}} F\left(x, y\right)\,{d x} $


<IPython.core.display.Math object>

x_sum=$ \frac{1}{2} \, F\left(n_{x}, y\right) $


<IPython.core.display.Math object>

x_R_p=$ \int_{0}^{1} {\sum_{j=n_{x}}^{b_{x} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \mathrm{D}_{0}\left(F\right)\left(j + v, y\right)}\,{d v} $


<IPython.core.display.Math object>

In [23]:
int_int = integral(integral           (F(x,y),(x, a_x, b_x)),  (y, a_y, b_y))
print("int_int=", int_int)
print("int_int=$", latex(int_int),"$")
display(Math(latex(int_int)))

int_sum = integral(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), (y, a_y, b_y))
print("int_sum=", int_sum)
print("int_sum=$", latex(int_sum.expand()),"$")
display(Math(latex(int_sum.expand())))

sumx_str = sum_dfdx_bernoulis_str (F(x,y), x, a_x, b_x, p)
inty_sumx_str = "integral(" + sumx_str + ", (y, a_y, b_y))"
print("inty_sumx_str=", inty_sumx_str)
exec(preparse("inty_sumx=" + inty_sumx_str))
print(preparse("inty_sumx=" + inty_sumx_str))
print("inty_sumx = $", latex(inty_sumx),"$")
display(Math("inty\_sumx = " + latex(inty_sumx)))

R_px = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p)
if type(R_px) is tuple:
    inty_R_px = integral(R_px[0], (y, a_y, b_y), hold=True)
else:
    inty_R_px = integral(R_px, (y, a_y, b_y), hold=True)

print("inty_R_px=",inty_R_px)
print("inty_R_px=$",latex(inty_R_px),"$")
display(Math(latex(inty_R_px)))

inty_R_px = integral_euler_maclaurin_R_p(F(x,y), x, a_x, b_x, y, a_y, b_y, p, diff(F(x,y),x,p), hold_int=True, hold_sum=True)
print("inty_R_px=",inty_R_px)
print("inty_R_px=$",latex(inty_R_px),"$")
display(Math(latex(inty_R_px)))


<>:17: DeprecationWarning: invalid escape sequence \_
<>:17: DeprecationWarning: invalid escape sequence \_
<ipython-input-23-56a4512e2a50>:17: DeprecationWarning: invalid escape sequence \_
  display(Math("inty\_sumx = " + latex(inty_sumx)))


int_int= integrate(integrate(F(x, y), x, n_x, b_x), y, n_y, b_y)
int_int=$ \int_{n_{y}}^{b_{y}} \int_{n_{x}}^{b_{x}} F\left(x, y\right)\,{d x}\,{d y} $


<IPython.core.display.Math object>

int_sum= 1/2*integrate(F(n_x, y), y, n_y, b_y)
int_sum=$ \frac{1}{2} \, \int_{n_{y}}^{b_{y}} F\left(n_{x}, y\right)\,{d y} $


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

inty_R_px= integrate(integrate(sum(1/2*(2*v - 1)*D[0](F)(j + v, y), j, n_x, b_x - 1), v, 0, 1), y, n_y, b_y)
inty_R_px=$ \int_{n_{y}}^{b_{y}} \int_{0}^{1} {\sum_{j=n_{x}}^{b_{x} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \mathrm{D}_{0}\left(F\right)\left(j + v, y\right)}\,{d v}\,{d y} $


<IPython.core.display.Math object>

inty_R_px= integrate(integrate(sum(1/2*(2*v - 1)*D[0](F)(j + v, y), j, n_x, b_x - 1), v, 0, 1), y, n_y, b_y)
inty_R_px=$ \int_{n_{y}}^{b_{y}} \int_{0}^{1} {\sum_{j=n_{x}}^{b_{x} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \mathrm{D}_{0}\left(F\right)\left(j + v, y\right)}\,{d v}\,{d y} $


<IPython.core.display.Math object>

In [24]:
sum_int = sum_dfdx_bernoulis(integral           (F(x,y),(x, a_x, b_x)),  y, a_y, b_y, p)
print("sum_int=",sum_int)
print("sum_int=$",latex(sum_int),"$")
display(Math(latex(sum_int)))

sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
print("sum_sum=",sum_sum)
print("sum_sum=$",latex(sum_sum),"$")
display(Math(latex(sum_sum)))

R_px = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, f_diff_symb_p=F(x,y).diff(x,p))
print("R_px=", R_px)
print("R_px=$",latex(R_px),"$")
display(Math(latex(R_px)))

sumy_R_px = sum_dfdx_bernoulis(R_px, y, a_y, b_y, p)
print("sumy_R_px=",sumy_R_px)
print("sumy_R_px=$",latex(sumy_R_px),"$")
display(Math(latex(sumy_R_px)))

sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, f_diff_symb_p=F(x,y).diff(x,p)), y, a_y, b_y, p)
print("sum_R_p=",sum_R_p)
print("sum_R_p=$",latex(sum_R_p),"$")
display(Math(latex(sum_R_p)))

sum_R_p = sum_dfdx_bernoulis_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p)

if type(sum_R_p) is tuple:
    print("sum_R_p=",sum_R_p[0])
    print("sum_R_p=$",latex(sum_R_p[0]),"$")
    display(Math(latex(sum_R_p[0])))
else:
    print("sum_R_p=",sum_R_p)
    print("sum_R_p=$",latex(sum_R_p),"$")
    display(Math(latex(sum_R_p)))

sum_int= 1/2*integrate(F(x, n_y), x, n_x, b_x)
sum_int=$ \frac{1}{2} \, \int_{n_{x}}^{b_{x}} F\left(x, n_{y}\right)\,{d x} $


<IPython.core.display.Math object>

sum_sum= 1/4*F(n_x, n_y)
sum_sum=$ \frac{1}{4} \, F\left(n_{x}, n_{y}\right) $


<IPython.core.display.Math object>

R_px= integrate(sum(1/2*(2*v - 1)*D[0](F)(j + v, y), j, n_x, b_x - 1), v, 0, 1)
R_px=$ \int_{0}^{1} {\sum_{j=n_{x}}^{b_{x} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \mathrm{D}_{0}\left(F\right)\left(j + v, y\right)}\,{d v} $


...

<IPython.core.display.Math object>


Can't calculate derivative wrt j_x + v_x.


<ipython-input-11-5c1c549f1026>:18: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  R_p = lambda y_symb, y_val : symbolic_sum(expression=term_1(y_symb=y_symb, y_val=y_val, jx=jx),                                                                      v=jx, a=a_x, b=b_x-Integer(1))



Can't calculate derivative wrt j_x + v_x.
sum_R_p= -1/4*sum(integrate(2*v_x*D[0](F)(j_x + v_x, b_y) - D[0](F)(j_x + v_x, b_y), v_x, 0, 1), j_x, n_x, b_x - 1) + 1/4*sum(integrate(2*v_x*D[0](F)(j_x + v_x, n_y) - D[0](F)(j_x + v_x, n_y), v_x, 0, 1), j_x, n_x, b_x - 1)
sum_R_p=$ -\frac{1}{4} \, {\sum_{j_{x}=n_{x}}^{b_{x} - 1} \int_{0}^{1} 2 \, v_{x} \mathrm{D}_{0}\left(F\right)\left(j_{x} + v_{x}, b_{y}\right) - \mathrm{D}_{0}\left(F\right)\left(j_{x} + v_{x}, b_{y}\right)\,{d v_{x}}} + \frac{1}{4} \, {\sum_{j_{x}=n_{x}}^{b_{x} - 1} \int_{0}^{1} 2 \, v_{x} \mathrm{D}_{0}\left(F\right)\left(j_{x} + v_{x}, n_{y}\right) - \mathrm{D}_{0}\left(F\right)\left(j_{x} + v_{x}, n_{y}\right)\,{d v_{x}}} $


<IPython.core.display.Math object>

In [25]:
R_p_int = euler_maclaurin_R_p(integral           (F(x,y),(x, a_x, b_x), hold=True), \
                              y, a_y, b_y, p,f_diff_symb_p=F(x,y).diff(y,p))
print("R_p_int=",R_p_int)
print("R_p_int=$",latex(R_p_int),"$")
display(Math(latex(R_p_int)))

R_p_sum = euler_maclaurin_R_p(sum_dfdx_bernoulis (F(x,y), x, a_x, b_x,p), \
                              y, a_y, b_y, p, f_diff_symb_p=F(x,y).diff(y,p))
print("R_p_sum=",R_p_sum)
print("R_p_sum=$",latex(R_p_sum),"$")
display(Math(latex(R_p_sum)))

try:
    R_px = euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p, f_diff_symb_p=F(x,y).diff(x,p))
    print("R_px=",R_px)
    print("R_px=$",latex(R_px),"$")
    display(Math(latex(R_px)))
    
    R_px_diff_y_p = R_px.diff(y,p)
    print("R_px_diff_y_p=", R_px_diff_y_p)
    print("R_px_diff_y_p=$", latex(R_px_diff_y_p),"$")
    display(Math(latex(R_px_diff_y_p)))
    
    logging = True

    R_p_R_p = euler_maclaurin_R_p(R_px, \
                                  y, a_y, b_y, p, f_diff_symb_p=R_px_diff_y_p)
    #R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p)
    
    logging = False

    print("R_p_R_p=",R_p_R_p)
    print("R_p_R_p=$",latex(R_p_R_p),"$")
    display(Math(latex(R_p_R_p)))
except Exception as ex:
    print(ex)


try:
    #R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
    R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p, f_diff_x_p_diff_y_p=None)

    print("R_p_R_p=",R_p_R_p)
    print("R_p_R_p=$",latex(R_p_R_p),"$")
    display(Math(latex(R_p_R_p)))
except Exception as ex:
    print(ex)


Can't calculate derivative wrt j + v.
R_p_int= integrate(sum(1/2*(2*v - 1)*integrate(D[1](F)(x, j + v), x, n_x, b_x), j, n_y, b_y - 1), v, 0, 1)
R_p_int=$ \int_{0}^{1} {\sum_{j=n_{y}}^{b_{y} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \int_{n_{x}}^{b_{x}} \mathrm{D}_{1}\left(F\right)\left(x, j + v\right)\,{d x}}\,{d v} $


<IPython.core.display.Math object>

R_p_sum= integrate(sum(1/4*(2*v - 1)*D[1](F)(n_x, j + v), j, n_y, b_y - 1), v, 0, 1)
R_p_sum=$ \int_{0}^{1} {\sum_{j=n_{y}}^{b_{y} - 1} \frac{1}{4} \, {\left(2 \, v - 1\right)} \mathrm{D}_{1}\left(F\right)\left(n_{x}, j + v\right)}\,{d v} $


<IPython.core.display.Math object>

R_px= integrate(sum(1/2*(2*v - 1)*D[0](F)(j + v, y), j, n_x, b_x - 1), v, 0, 1)
R_px=$ \int_{0}^{1} {\sum_{j=n_{x}}^{b_{x} - 1} \frac{1}{2} \, {\left(2 \, v - 1\right)} \mathrm{D}_{0}\left(F\right)\left(j + v, y\right)}\,{d v} $


...

<IPython.core.display.Math object>

f integrate(sum(1/2*(2*v - 1)*D[0](F)(j + v, y), j, n_x, b_x - 1), v, 0, 1)
symb,a,b y n_y b_y

R_p_R_p= integrate(sum(0, j, n_y, b_y - 1), v, 0, 1)
R_p_R_p=$ \int_{0}^{1} {\sum_{j=n_{y}}^{b_{y} - 1} 0}\,{d v} $


<IPython.core.display.Math object>


Can't calculate derivative wrt j_x + v_x.
R_p_R_p= sum(integrate(1/4*(2*v_y - 1)*integrate((2*v_x - 1)*D[0, 1, 1](F)(j_x + v_x, j_y + v_y), v_x, 0, 1)*D[0](sum)(1/2*integrate((2*v_x - 1)*D[0, 1](F)(j_x + v_x, j_y + v_y), v_x, 0, 1), j_x, n_x, b_x - 1), v_y, 0, 1), j_y, n_y, b_y - 1)
R_p_R_p=$ {\sum_{j_{y}=n_{y}}^{b_{y} - 1} \int_{0}^{1} \frac{1}{4} \, {\left(2 \, v_{y} - 1\right)} \int_{0}^{1} {\left(2 \, v_{x} - 1\right)} \mathrm{D}_{0, 1, 1}\left(F\right)\left(j_{x} + v_{x}, j_{y} + v_{y}\right)\,{d v_{x}} \mathrm{D}_{0}{\sum_{j_{x}=n_{x}}^{b_{x} - 1} \frac{1}{2} \, \int_{0}^{1} {\left(2 \, v_{x} - 1\right)} \mathrm{D}_{0, 1}\left(F\right)\left(j_{x} + v_{x}, j_{y} + v_{y}\right)\,{d v_{x}}}\,{d v_{y}}} $


<IPython.core.display.Math object>

In [26]:
sxy_symb = euler_maclaurin1(euler_maclaurin1(F(x,y),x,a_x,b_x,p,f_diff_symb_p=None),\
                            y,a_y,b_y,p,f_diff_symb_p=None)


Can't calculate derivative wrt j + v.

Can't calculate derivative wrt j + v.


In [27]:
sxy_symb

1/4*F(n_x, n_y) + 1/4*integrate((2*v - 1)*sum(D[0](F)(j + v, n_y), j, n_x, b_x - 1), v, 0, 1) + integrate(1/2*F(n_x, y) + 1/2*integrate((2*v - 1)*sum(D[0](F)(j + v, y), j, n_x, b_x - 1), v, 0, 1) + integrate(F(x, y), x, n_x, b_x), y, n_y, b_y) + 1/2*integrate(F(x, n_y), x, n_x, b_x) + integrate(sum(1/4*(2*v - 1)*(2*integrate(D[1](F)(x, j + v), x, n_x, b_x) + D[1](F)(n_x, j + v)), j, n_y, b_y - 1), v, 0, 1)

In [28]:
latex(sxy_symb)

\frac{1}{4} \, F\left(n_{x}, n_{y}\right) + \frac{1}{4} \, \int_{0}^{1} {\left(2 \, v - 1\right)} {\sum_{j=n_{x}}^{b_{x} - 1} \mathrm{D}_{0}\left(F\right)\left(j + v, n_{y}\right)}\,{d v} + \int_{n_{y}}^{b_{y}} \frac{1}{2} \, F\left(n_{x}, y\right) + \frac{1}{2} \, \int_{0}^{1} {\left(2 \, v - 1\right)} {\sum_{j=n_{x}}^{b_{x} - 1} \mathrm{D}_{0}\left(F\right)\left(j + v, y\right)}\,{d v} + \int_{n_{x}}^{b_{x}} F\left(x, y\right)\,{d x}\,{d y} + \frac{1}{2} \, \int_{n_{x}}^{b_{x}} F\left(x, n_{y}\right)\,{d x} + \int_{0}^{1} {\sum_{j=n_{y}}^{b_{y} - 1} \frac{1}{4} \, {\left(2 \, v - 1\right)} {\left(2 \, \int_{n_{x}}^{b_{x}} \mathrm{D}_{1}\left(F\right)\left(x, j + v\right)\,{d x} + \mathrm{D}_{1}\left(F\right)\left(n_{x}, j + v\right)\right)}}\,{d v}

In [29]:
display(Math(latex(sxy_symb)))

<IPython.core.display.Math object>

In [30]:
f(x,y)=1/(x^2)
assume(y>0)
assume(x>0)
a_x = 1
b_x = Infinity

a_y = 1
b_y = Infinity

In [31]:
calc_R_p=True
use_int_R_p_j=False
use_int_R_p_shifted_first=True
change_R_p_order=True

In [32]:
ans = my_euler_maclaurin1(f(x,y), x, a_x, b_x, p, f_diff_symb_p=None)
ans

integrate(-(2*x - 2*floor(x) - 1)/x^3, x, 1, +Infinity) + 3/2

In [33]:
ans = euler_maclaurin1(f(x,y), x, a_x, b_x, p, f_diff_symb_p=None)
ans

integrate(-(2*x - 2*floor(x) - 1)/x^3, x, 1, +Infinity) + 3/2

In [34]:
ans.n()

1.64489926189016

In [35]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=False

In [36]:
ans = my_euler_maclaurin1(f,x,a_x,b_x,p, f_diff_symb_p=None)
ans

(x, y) |--> 1/6*pi^2

In [37]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None)
ans

sum(integrate(-(2*v_x - 1)/(j_x + v_x)^3, v_x, 0, 1), j_x, 1, +Infinity) + 3/2

In [38]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=True

In [39]:
ans = my_euler_maclaurin1(f,x,a_x,b_x,p, f_diff_symb_p=None)
ans

(x, y) |--> integrate(2*v*(harmonic_number(v, 3) - zeta(3)) - harmonic_number(v, 3) + zeta(3), v, 0, 1) + 3/2

In [40]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None)
ans

integrate(sum(-(2*v - 1)/(j + v)^3, j, 1, +Infinity), v, 0, 1) + 3/2

In [41]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None, hold_int=False)
ans

integrate(sum(-(2*v - 1)/(j + v)^3, j, 1, +Infinity), v, 0, 1) + 3/2

In [42]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=False

In [43]:
ans = my_euler_maclaurin1(f,x,a_x,b_x,p, f_diff_symb_p=None)
ans

(x, y) |--> 1/6*pi^2

In [44]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None)
ans

sum(integrate(-(2*v_x - 1)/(j_x + v_x)^3, v_x, 0, 1), j_x, 1, +Infinity) + 3/2

In [45]:
ans = euler_maclaurin1(f(x,y),x,a_x,b_x,p, f_diff_symb_p=None, hold_int=False)
ans

sum(1/2*(2*j_x + 3)/(j_x^2 + 2*j_x + 1) - 1/2*(2*j_x - 1)/j_x^2, j_x, 1, +Infinity) + 3/2

In [98]:
f(x,y)=1/((x^2+y^2)^2)
#f(x,y)=x^2+y^2
assume(y>0)
assume(x>0)

In [99]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=False
change_R_p_order=False

In [ ]:
#sxy1 = euler_maclaurin1(euler_maclaurin1(f(x,y),x,a_x,b_x,p),y,a_y,b_y,p)

In [ ]:
#display(Math(latex(my_sxy1)))

In [50]:
#display(Math(latex(sxy1)))

In [51]:
#my_sxy1.n()

In [52]:
#sxy1.n()

In [53]:
#display(Math(latex(sum(1/(x^2+y^2)^2,x,1,Infinity))))

In [54]:
#display(Math(latex(sum(sum(1/(x^2+y^2)^2,x,1,Infinity),y,1,Infinity))))

In [95]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=False
change_R_p_order=False

use_int_R_p_j = False
use_num_int = False

int_int = integral(integral           (f(x,y),(x, a_x, b_x)),  (y, a_y, b_y)) 
print("int_int=$", latex(int_int),"$")
print("int_int=", int_int.n())
display(Math(latex(int_int)))

int_sum = integral(sum_dfdx_bernoulis (f(x,y), x, a_x, b_x,p), (y, a_y, b_y)) 
print("int_sum=$", latex(int_sum.expand()),"$")
print("int_sum=", int_sum.n())
display(Math(latex(int_sum.expand())))

try:
    R_p = euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p, f_diff_symb_p=None)
    if type(R_p) is tuple:
        int_R_p = integral(R_p[0], (y, a_y, b_y), hold=True)
    else:
        int_R_p = integral(R_p, (y, a_y, b_y), hold=True)

    print("int_R_p=$",latex(int_R_p),"$")
    #print("int_R_p=", int_R_p.n())
    display(Math(latex(int_R_p)))
except Exception as ex:
    print(ex)

use_num_int = True
f_diff_x_p_str = str(f(x,y).diff(x,p))
print("f_diff_x_p_str=", f_diff_x_p_str)
exec(preparse("f_diff_x_p = " + f_diff_x_p_str))
print(preparse("f_diff_x_p = " + f_diff_x_p_str))
print("f_diff_x_p = $", latex(f_diff_x_p),"$")

int_R_p_ans = integral_euler_maclaurin_R_p(f(x,y), x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p=f_diff_x_p, hold_int=False)
if type(int_R_p_ans) is tuple:
    (int_R_p,int_int_R_p) = int_R_p_ans
else:
    int_R_p = int_R_p_ans

print("int_R_p=$",latex(int_R_p),"$")
print("int_R_p=",int_R_p)
#print("int_R_p=", int_R_p.n())
display(Math(latex(int_R_p)))

int_int=$ \frac{1}{8} \, \pi - \frac{1}{4} $
int_int= 0.142699081698724


<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2/(x^2 + y^2)^2
sum_dfdx_a_bernoullis(a) -1/2/(y^2 + 1)^2
int_sum=$ \frac{1}{16} \, \pi - \frac{1}{8} $
int_sum= 0.0713495408493621


<IPython.core.display.Math object>

f (x^2 + y^2)^(-2)
symb,a,b x 1 +Infinity

int_R_p=$ \int_{1}^{+\infty} \int_{1}^{+\infty} -\frac{2 \, {\left(2 \, x - 2 \, \left \lfloor x \right \rfloor - 1\right)} x}{{\left(x^{2} + y^{2}\right)}^{3}}\,{d x}\,{d y} $


<IPython.core.display.Math object>

f_diff_x_p_str= -4*x/(x^2 + y^2)^3
f_diff_x_p = -Integer(4)*x/(x**Integer(2) + y**Integer(2))**Integer(3)
f_diff_x_p = $ -\frac{4 \, x}{{\left(x^{2} + y^{2}\right)}^{3}} $
f (x^2 + y^2)^(-2)
f(x,y) (x^2 + y^2)^(-2)
f_diff_x_p -4*x/(x^2 + y^2)^3
f_diff_x_p(x,y) -4*x/(x^2 + y^2)^3



<ipython-input-95-b3b631bf5fcf>:40: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  int_R_p_ans = integral_euler_maclaurin_R_p(f(x,y), x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p=f_diff_x_p, hold_int=False)
<ipython-input-1-b7de2e928668>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral = integrate.quad(f, a, b)
<ipython-input-1-b7de2e928668>:5: IntegrationWarning: The in

int_R_p=$ 0.014591148758590414 $
int_R_p= 0.014591148758590414


<IPython.core.display.Math object>

In [56]:
int_int

1/8*pi - 1/4

In [57]:
int_int.n()

0.142699081698724

In [58]:
int_sum

1/16*pi - 1/8

In [59]:
int_sum.n()

0.0713495408493621

In [60]:
int_R_p

0.014591148758590414

In [61]:
#int_R_p.n()

In [62]:
try:
    print(sum(int_int_R_p, j_x, 1, +Infinity))
except Exception as ex:
    print(ex)

name 'int_int_R_p' is not defined


In [63]:
try:
    int_R_p_s = 0
    for j in range (1, 1000):
        int_R_p_term = int_int_R_p.subs(var("j_x") == j)
        int_R_p_s += int_R_p_term
        #print(j, int_R_p_term.n(), int_R_p_n.n())
    int_R_p_n = int_R_p_s.n()
    print(int_R_p_n)
except Exception as ex:
    print(ex)
    int_R_p_n = int_R_p
    print(int_R_p_n)

name 'int_int_R_p' is not defined
0.014591148758590414


In [65]:
logging = True
sum_int = sum_dfdx_bernoulis(integral           (f(x,y),(x, a_x, b_x)),  y, a_y, b_y, p)
print("sum_int=$",latex(sum_int),"$")
print("sum_int=",sum_int.n())
display(Math(latex(sum_int)))

sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (f(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
print("sum_sum=$",latex(sum_sum),"$")
print("sum_sum=", sum_sum.n())

display(Math(latex(sum_sum)))

#sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
(sum_R_p, dfdx_a_bernoullis_terms) = sum_dfdx_bernoulis_euler_maclaurin_R_p(f(x,y), x, a_x, b_x,    y, a_y, b_y, p)
print("sum_R_p=$",latex(sum_R_p),"$")
#print("sum_R_p=$",sum_R_p.n())
display(Math(latex(sum_R_p)))

sum_dfdx_a_bernoullis 1/4*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3) - 1/8*pi/y^3
sum_dfdx_a_bernoullis(a) -1/16*pi + 1/8
sum_int=$ \frac{1}{16} \, \pi - \frac{1}{8} $
sum_int= 0.0713495408493621


<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2/(x^2 + y^2)^2
sum_dfdx_a_bernoullis(a) -1/2/(y^2 + 1)^2
sum_dfdx_a_bernoullis -1/4/(y^2 + 1)^2
sum_dfdx_a_bernoullis(a) -1/16
sum_sum=$ \frac{1}{16} $
sum_sum= 0.0625000000000000


<IPython.core.display.Math object>

sum_dfdx_bernoulis_euler_maclaurin_R_p (x^2 + y^2)^(-2) x 1 +Infinity y 1 +Infinity 1
term_1(y_symb=y, y_val=a_y) = -2*integrate((2*x - 2*floor(x) - 1)*x/(x^2 + 1)^3, x, 1, +Infinity)
R_p(y_symb=y, y_val=a_y) = -2*integrate((2*x - 2*floor(x) - 1)*x/(x^2 + 1)^3, x, 1, +Infinity)
dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7f7a0969f940>]
len(dfdx_a_bernoullis) = 1
sum_dfdx_a_bernoullis = <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7f7a0969f280>
sum_dfdx_a_bernoullis(a_y) = integrate((2*x - 2*floor(x) - 1)*x/(x^2 + 1)^3, x, 1, +Infinity)
sum_R_p=$ -\int_{1}^{+\infty} \frac{{\left(2 \, x - 2 \, \left \lfloor x \right \rfloor - 1\right)} x}{{\left(x^{2} + 1\right)}^{3}}\,{d x} $


<ipython-input-11-5c1c549f1026>:23: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  print("term_1(y_symb=y, y_val=a_y) =", term_1(y_symb=y, y_val=a_y))
<ipython-input-11-5c1c549f1026>:24: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  R_p = lambda y_symb, y_val : (-Integer(1))**(p+Integer(1))*term_1(y_symb, y_val)


<IPython.core.display.Math object>

In [66]:
sum_R_p

-integrate((2*x - 2*floor(x) - 1)*x/(x^2 + 1)^3, x, 1, +Infinity)

In [67]:
if use_int_R_p_j:
    print(dfdx_a_bernoullis_terms(j_x))

In [68]:
if use_int_R_p_j:
    sum_R_p_n = 0
    for j_x in range(1,100):
        term = dfdx_a_bernoullis_terms(j_x)
        sum_R_p_n += term
        #print(j_x, term.n(), sum_R_p_n)
    print(sum_R_p_n.n())

In [70]:
use_int_R_p_j = False
use_num_int = False
R_p_int = euler_maclaurin_R_p(integral           (f(x,y),(x, a_x, b_x), hold = False),  y, a_y, b_y, p, f_diff_symb_p=None)
print("R_p_int=$",latex(R_p_int),"$")
#print("R_p_int=", R_p_int.n())
display(Math(latex(R_p_int)))

R_p_sum = euler_maclaurin_R_p(sum_dfdx_bernoulis (f(x,y), x, a_x, b_x,p), y, a_y, b_y, p, f_diff_symb_p=None, hold_int=False)
print("R_p_sum=$",latex(R_p_sum),"$")
#print("R_p_sum=$",R_p_sum.n())
display(Math(latex(R_p_sum)))


f -1/2*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3) + 1/4*pi/y^3
symb,a,b y 1 +Infinity

R_p_int=$ \int_{1}^{+\infty} \frac{1}{8} \, {\left(2 \, y - 2 \, \left \lfloor y \right \rfloor - 1\right)} {\left(\frac{2 \, {\left(5 \, y^{4} + 3 \, y^{2}\right)} {\left(y^{2} \arctan\left(\frac{1}{y}\right) + y + \arctan\left(\frac{1}{y}\right)\right)}}{{\left(y^{5} + y^{3}\right)}^{2}} - \frac{2 \, {\left(2 \, y \arctan\left(\frac{1}{y}\right) - \frac{1}{\frac{1}{y^{2}} + 1} - \frac{1}{y^{2} {\left(\frac{1}{y^{2}} + 1\right)}} + 1\right)}}{y^{5} + y^{3}} - \frac{3 \, \pi}{y^{4}}\right)}\,{d y} $


<IPython.core.display.Math object>

sum_dfdx_a_bernoullis -1/2/(x^2 + y^2)^2
sum_dfdx_a_bernoullis(a) -1/2/(y^2 + 1)^2
f 1/2/(y^2 + 1)^2
symb,a,b y 1 +Infinity

R_p_sum=$ -\int_{1}^{+\infty} \frac{{\left(2 \, y - 2 \, \left \lfloor y \right \rfloor - 1\right)} y}{{\left(y^{2} + 1\right)}^{3}}\,{d y} $


<IPython.core.display.Math object>

In [71]:
R_p_int

integrate(1/8*(2*y - 2*floor(y) - 1)*(2*(5*y^4 + 3*y^2)*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^2 - 2*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3) - 3*pi/y^4), y, 1, +Infinity)

In [72]:
integral           (f(x,y),(x, a_x, b_x), hold = False)

-1/2*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3) + 1/4*pi/y^3

In [73]:
R_p_int.n()

0.014593410520966658

In [74]:
R_p_sum

-integrate((2*y - 2*floor(y) - 1)*y/(y^2 + 1)^3, y, 1, +Infinity)

In [75]:
R_p_sum.n()

0.0195690550052681

In [76]:
#use_num_int = True

#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
#R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(f(x,y), x, a_x, b_x,    y, a_y, b_y, p)
#print("R_p_R_p=$",latex(R_p_R_p),"$")
#print("R_p_R_p=$", R_p_R_p.n())
#display(Math(latex(R_p_R_p)))

In [77]:
exec(preparse("f_diff_x_p_diff_y_p = lambda x,y:" + str(f(x,y).diff(x,p).diff(y,p))))
print("f_diff_x_p_diff_y_p", f_diff_x_p_diff_y_p(x,y))
R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p_num_int(f_diff_x_p_diff_y_p, x, a_x, b_x, y, a_y, b_y, p)

f_diff_x_p_diff_y_p 24*x*y/(x^2 + y^2)^4


<ipython-input-1-b7de2e928668>:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral = integrate.quad(f, a, b)
<ipython-input-1-b7de2e928668>:5: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = integrate.quad(f, a, b)


In [78]:
R_p_R_p

0.008150899241692044

# Summary Euler–Maclaurin 2D for $\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}$

In [79]:
int_int, int_sum, int_R_p

(1/8*pi - 1/4, 1/16*pi - 1/8, 0.014591148758590414)

In [80]:
sum_int, sum_sum, sum_R_p

(1/16*pi - 1/8,
 1/16,
 -integrate((2*x - 2*floor(x) - 1)*x/(x^2 + 1)^3, x, 1, +Infinity))

In [81]:
R_p_int, R_p_sum, R_p_R_p

(integrate(1/8*(2*y - 2*floor(y) - 1)*(2*(5*y^4 + 3*y^2)*(y^2*arctan(1/y) + y + arctan(1/y))/(y^5 + y^3)^2 - 2*(2*y*arctan(1/y) - 1/(1/y^2 + 1) - 1/(y^2*(1/y^2 + 1)) + 1)/(y^5 + y^3) - 3*pi/y^4), y, 1, +Infinity),
 -integrate((2*y - 2*floor(y) - 1)*y/(y^2 + 1)^3, y, 1, +Infinity),
 0.008150899241692044)

Numerical Euler–Maclaurin 2D for $\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}$

In [82]:
int_int.n(), int_sum.n(), int_R_p_n

(0.142699081698724, 0.0713495408493621, 0.014591148758590414)

In [83]:
sum_int.n(), sum_sum.n(), sum_R_p.n()

(0.0713495408493621, 0.0625000000000000, 0.0195690550052681)

In [84]:
R_p_int.n(), R_p_sum.n(), R_p_R_p

(0.014593410520966658, 0.0195690550052681, 0.008150899241692044)

In [85]:
int_int.n() + int_sum.n() + int_R_p_n + \
sum_int.n() + sum_sum.n() + sum_R_p.n() + \
R_p_int.n() + R_p_sum.n() + R_p_R_p

0.424371731929234

$\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}$

In [86]:
def zeta_2D_1_1(S, N):
    
    def zeta_2D_row(S, first, N):
        #print("N", N, [m for m in range(0,N)])
        row_sum = 0
        for m in range(first, N):
            row_sum += (N^2+m^2)^(-S)
            #print("m", m, "row_sum=", row_sum)
        return row_sum

    def zeta_2D_diag(S, N):
        diag_elem = (2*N^2)^(-S)
        #print("N", N, "diag_elem=", diag_elem)
        return diag_elem
    
    zeta_2D_sum = 0
    for n in range(1,N):
        zeta_2D_add = 2 * zeta_2D_row(S, 1, n) + zeta_2D_diag(S, n)
        zeta_2D_sum += zeta_2D_add
        
        #print(n, zeta_2D_add, zeta_2D_sum)
    return zeta_2D_sum

In [87]:
zeta_2D_1_1(2, 1000)

0.4243791332035987

R_p=False (Euler–Maclaurin 2D without remainer part)

In [88]:
int_int.n() + int_sum.n() + \
sum_int.n() + sum_sum.n()

0.347898163397448

# Two Dimensional Linear Zeta Function

https://core.ac.uk/download/pdf/82162715.pdf

JOURNAL OF NUMBER THEORY 35, 62-71 (1990)

Evaluation of Multidimensional Linear Zeta Functions

ALFRED ACTOR*

Department of Physics, The Pennsylvania State University,

Fogelsville, Pennsylvania 18051

Communicated by Hans Zassenhaus

Received January 28, 1988; revised May 5, 1989

A prototypical formula of this type is [1] 

$\sum\limits_{m,n=-\infty}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = 4\,\zeta\left(S\right)\,\beta\left(S\right)$

where of course, $\zeta\left(S\right)$ is the Riemann $\zeta$-function [2]

while $\beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^n \left(2n + 1\right)^{-S}$
defines, for $Res > 0$, a function whose properties throughout the s-plane are
well known [3].

Thus Eq. (1) fully reveals the properties of the twodimensional Epstein $\zeta$-function [4] on the left. 



1. G. H. HARDY, Notes on some points in the integral calculus LII, Mess. Math. 49 (1919),
85-91.
2. H. M. EDWARDS, “Riemann’s Zeta Function,” Academic Press, New York, 1974.
3. A. ERDELYI, W. MAGNUS, F. OEERHETTINGER, AND F. G. TRICOMI (Eds.), “Higher Transcendental Functions,” Vol. 1, McGraw-Hill, New York, 1953. 
LINEAR ZETA FUNCTIONS 71
4. P. EPSTEIN, Zur Theorie allgemeiner Zetafunctionen, Marh. Ann. 56 (1903), 615-645; Zur
Theorie allgemeiner Zetafunctionen, II, Math. Ann. 63 (19071, 205-216. 

$\sum\limits_{m,n=(0)1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = \zeta\left(S\right)\,\beta\left(S\right)$

$\sum\limits_{m,n=(0)1}^{\infty} \, \left(\sqrt{m^{2} + n^{2}}\right)^{-S} = \sum\limits_{m,n=(0)1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-\frac{S}{2}} = \zeta\left(\frac{S}{2}\right)\,\beta\left(\frac{S}{2}\right)$

In [89]:
def zeta_2D(S, N):
    
    def zeta_2D_row(S, first, N):
        #print("N", N, [m for m in range(0,N)])
        row_sum = 0
        for m in range(first, N):
            row_sum += (N^2+m^2)^(-S)
            #print("m", m, "row_sum=", row_sum)
        return row_sum

    def zeta_2D_diag(S, N):
        diag_elem = (2*N^2)^(-S)
        #print("N", N, "diag_elem=", diag_elem)
        return diag_elem
    
    zeta_2D_sum = 0
    for n in range(1,N):
        zeta_2D_add = zeta_2D_row(S, 0, n) + zeta_2D_row(S, 1, n) + zeta_2D_diag(S, n)
        zeta_2D_sum += zeta_2D_add
        
        #print(n, zeta_2D_add, zeta_2D_sum)
    return zeta_2D_sum

$\beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^n \left(2n + 1\right)^{-S}$

In [90]:
from sage.calculus.calculus import symbolic_sum
n = var("n")
S = var("S")
beta1_sym = lambda S : symbolic_sum((-1)^n * (2*n + 1)^(-S), n, 0, Infinity)
beta1 = lambda S : sum((-1)^n * (2*n + 1)^(-S), n, 0, Infinity)

def beta1_dig(S,N):
    Sum = 0
    for n in range (N):
        Sum += (-1)^n * (2*n + 1)^(-S)
    return Sum

$\sum\limits_{m,n=0,1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}=\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}+\zeta(2 S)$

$\sum\limits_{m,n=0,1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}=\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}+\zeta(4)$

In [91]:
int_int.n() + int_sum.n() + int_R_p_n + \
sum_int.n() + sum_sum.n() + sum_R_p.n() + \
R_p_int.n() + R_p_sum.n() + R_p_R_p + \
zeta(4).n()

1.50669496564037

$\frac{1}{4}\sum\limits_{m,n=-\infty}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = \,\zeta\left(S\right)\,\beta\left(S\right)$

In [92]:
beta1_dig(2,1000)*zeta(2).n()

1.50670280430638

In [93]:
zeta_2D(2, 1000)

1.5067023665809038